In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)



In [ ]:
worksheet = gc.open('Lista de invitados').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
import pandas as pd
lista=pd.DataFrame.from_records(rows)
lista.columns = lista.iloc[0]
lista=lista.drop(lista.index[0])


In [ ]:
lista.head()

,Invitation Type,Nombre destinatario,Apellido destinatario,Apellidos Familia,Telefono,Adultos,Niños,Total,Novio que invita,SobreNombre,Sexo,Relacion,KeyTexto,Enviado,Texto WhatsApp
1,F A M I L I A,Lucy,Pulido Morales,Sánchez Pulido,7228447178,6,0,6,Rebe,Familia Sanchez Pulido,F,Familiar,Familiar familia completa,1,"Estimada Familia Sanchez Pulido, te escribo co..."
2,I N V I T A D O,Cecilia,Morales Hernández,,5573345149,1,0,1,Rebe,Abuelita Cecy,F,Amigo,Amigo acompañante,0,"Estimada Abuelita Cecy, te escribo con mucho c..."
3,F A M I L I A,Georgina,Pulido Morales,López Pulido,5518562348,5,0,5,Rebe,Familia López Pulido,F,Familiar,Familiar familia completa,1,"Estimada Familia López Pulido, te escribo con ..."
4,F A M I L I A,Rubén,Pulido Morales,Pulido Osorio,5546364547,3,0,3,Rebe,Familia Pulido Osorio,M,Familiar,Familiar familia completa,0,"Estimado Familia Pulido Osorio, te escribo con..."
5,F A M I L I A,Saúl,Pulido Osorio,Pulido Téllez,,2,2,4,Rebe,Familia Pulido Téllez,M,Familiar,Familiar familia completa,0,"Estimado Familia Pulido Téllez, te escribo con..."


In [ ]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 208 kB 5.4 MB/s 


In [ ]:
!pip install reportlab

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.8 MB 5.2 MB/s 
     |████████████████████████████████| 3.1 MB 60.3 MB/s 
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2


In [ ]:
!pip install pdfplumber

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 40 kB 2.0 MB/s 
     |████████████████████████████████| 142 kB 8.3 MB/s 
     |████████████████████████████████| 5.6 MB 50.6 MB/s 
     |████████████████████████████████| 4.1 MB 24.7 MB/s 


In [ ]:
from PyPDF2 import PdfFileWriter, PdfFileReader
import io
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch

from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

pdfmetrics.registerFont(TTFont('Montserrat', 'Montserrat-Light.ttf'))
pdfmetrics.registerFont(TTFont('Allura', 'Allura-Regular.ttf'))


In [ ]:
for invitado in range(0,len(lista)):
#for invitado in range(0,3):

  Familia = lista.iloc[invitado,0]
  Nombre = lista.iloc[invitado,1]
  Apellidos = lista.iloc[invitado,2]
  ApellidosFamilia = lista.iloc[invitado,3]
  Boletos = lista.iloc[invitado,7]

  packet = io.BytesIO()
  can = canvas.Canvas(packet, pagesize=letter)

  # define a large font
  can.setFont("Allura", 20)
  # choose some colors
  can.setStrokeColorRGB(255,255,255)
  can.setFillColorRGB(255,255,255)
  # draw a rectangle
  can.rect(30,40,350,50, fill=1)
  # change color
  can.setFillColorRGB(0,0,0)
  if Familia=="F A M I L I A":
      can.setFont("Allura", 20)
      can.drawString(180, 65, ApellidosFamilia)
  else: 
    if len(Nombre+Apellidos)<=20:
      can.setFont("Allura", 20)
      can.drawString(200, 65, Nombre+ " " +Apellidos)
    else:
      can.setFont("Allura", 18)
      can.drawString(200, 75, Nombre)
      can.drawString(200, 58, Apellidos)

  # change color
  can.setFont("Montserrat", 20)
  can.setFillColorRGB(0.78,0.72,0.70)
  can.setStrokeColorRGB(0.78,0.72,0.70)
  can.drawString(50, 65, Familia+" :")
  # change color
  can.setFont("Montserrat", 12)
  can.setFillColorRGB(0,0,0)
  can.setStrokeColorRGB(0,0,0)

  if Familia=="F A M I L I A":
     can.drawString(180, 40, Boletos+"   B O L E T O S")
  else: 
    if Boletos=="1":
      can.drawString(200, 40, Boletos+"   B O L E T O")
    else: 
      can.drawString(200, 40, Boletos+"   B O L E T O S")


  can.showPage()
  can.save()

  #move to the beginning of the StringIO buffer
  packet.seek(0)

  # create a new PDF with Reportlab
  new_pdf = PdfFileReader(packet)
  # read your existing PDF
  existing_pdf = PdfFileReader(open("/content/PDF.pdf", "rb"))
  output = PdfFileWriter()
  # add the "watermark" (which is the new pdf) on the existing page
  page = existing_pdf.getPage(0)
  page.mergePage(new_pdf.getPage(0))
  output.addPage(page)
  # finally, write "output" to a real file
  outputStream = open("Invitación" +" "+Nombre+" "+ Apellidos+".pdf", "wb")
  output.write(outputStream)
  outputStream.close()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import shutil

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
path = r"/content/"   
invitaciones=[x for x in os.listdir(path) if "Invitación" in x]
for x in invitaciones:
  shutil.copy("/content/"+x,"/content/drive/MyDrive/Invitaciones")
